In [1]:
#### librerie ########
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re 


In [2]:
###inizio scraping prima lista "Accesso con Laurea" 
# Impostare il percorso dell'eseguibile di ChromeDriver
driver = webdriver.Chrome()

# Aprire la pagina desiderata
driver.get('https://www.universitaly.it/index.php/cercacorsi/universita#null')

# Selezionare il radio button "Accesso con Laurea"
accesso_radio = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'input#accesso_2'))
)
accesso_radio.click()

# Accettare i cookie, se presenti
try:
    cookie_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.cc_btn_accept_all'))
    )
    cookie_button.click()
except:
    pass  # se non ci sono i cookie, si passa semplicemente avanti

# attende che il bottone dei cookie sia cliccabile e lo clicca
cookie_button = WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.cc_btn_accept_all[data-cc-event="click:dismiss"]'))
)
cookie_button.click()


# Clicca sul pulsante "Cerca"
cerca_button = driver.find_element(By.CSS_SELECTOR,'input#avvia_ricerca.btn-form')
cerca_button.click()

print('sono qui 1')
time.sleep(5)
print('sono qui 2')

# Aspettiamo che i risultati vengano caricati
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#risultati > div.skin > div > table > tbody > tr')))

# seleziona tutte le righe della tabella dei risultati
rows = driver.find_elements_by_xpath("//a[contains(text(), 'scheda completa (SUA-CDS)')]")

link_list = []
text_list = []
lm_list = []
course_list = []
uni_list = []
accesso = []
numero_chiuso = []
presenza = []

#we have now the list of all innermost tags. We can loop through it and for each element we check if it's null or not
#and the we append its value to the relevant list
for row in rows:
    
    #get the link in the innermost tag
    link = row.get_attribute('href')
    if len(link) == 0:
        link_list.append('0')
    else:
        link_list.append(link)
    
    #jump one level above from the child tag and look for the text
    parent_text = row.find_element_by_xpath("..").text
    if len(parent_text) == 0:
        text_list.append('0')
    else:
        text_list.append(parent_text)
    
    #jump 2 levels above, search for a specific td class (named 'classe') where the relevant info is stored
    #and return its text
    parent_tag = row.find_element_by_xpath("../..")
    lm_text = parent_tag.find_element_by_xpath(".//td[@class='classe']").text
    if len(lm_text) == 0:
        lm_list.append('0')
    else:
        lm_list.append(lm_text)
    
    #look for additional attribute at the same level of parent_tag
    attribute_tag = parent_tag.find_elements_by_xpath(".//td[@class='icona']")
    if len(attribute_tag[0].find_element_by_tag_name("img").get_attribute('title')) == 0:
        accesso.append('0')
    else:
        accesso.append(attribute_tag[0].find_element_by_tag_name("img").get_attribute('title'))
    
    if len(attribute_tag[2].find_element_by_tag_name("img").get_attribute('title')) == 0:
        numero_chiuso.append('0')
    else:
        numero_chiuso.append(attribute_tag[2].find_element_by_tag_name("img").get_attribute('title'))

    if len(attribute_tag[3].find_element_by_tag_name("img").get_attribute('title')) == 0:
        presenza.append('0')
    else:
        presenza.append(attribute_tag[3].find_element_by_tag_name("img").get_attribute('title'))
    
    #add the link to the course website
    tag_1 = row.find_element_by_xpath("../..")
    course_tag = tag_1.find_element_by_xpath(".//a[contains(text(),'sito del corso')]")
    course_link = course_tag.get_attribute('href')
    if len(course_link) == 0:
        course_list.append('0')
    else:
        course_list.append(course_link)



df = pd.DataFrame({'Link':link_list,'descrizione':text_list,'corso_url': course_list,'LM_CODE':lm_list, 'Presenza':presenza, 'Numero_Chiuso':numero_chiuso, 'Accesso':accesso})


# Chiudere il browser
driver.quit()

sono qui 1
sono qui 2


In [3]:
#adding pdf_id
numbers = []

for item in df['Link']:
    a = int(item[-5:])
    numbers.append(a)
    
##adding city
city = []
for item in text_list:
    y = re.findall('[A-Z]+\n', item)
    if len(y) == 0:
        city.append('FORLI')
    else:
        nn = y[0].find('/')
        city.append(y[0][:nn])
        
df['PDF_ID'] = numbers
df['Cittá'] = city

In [4]:
#find university url from course link
pattern = r"(\w+)\.it"
ateneo = []

for url in df['corso_url']:
    match = re.search(pattern, url)
    if match:
        preceding_word = match.group(1)
        ateneo.append('www.'+ preceding_word + '.it')
    else:
        ateneo.append('check corso_url')

df['ateneo_url'] = ateneo

In [7]:
#create raw data file to process in AWS
df.to_csv('raw_data_master.csv')

,Link,descrizione,corso_url,LM_CODE,Presenza,Numero_Chiuso,Accesso,PDF_ID,Cittá,ateneo_url
0,https://www.universitaly.it/index.php/scheda/s...,"Archeologia , BARI\nInterateneo\n[scheda compl...",https://www.uniba.it/corsi/archeologia,LM-2,in presenza,Libero,Accesso con laurea,59932,BARI,www.uniba.it
1,https://www.universitaly.it/index.php/scheda/s...,"Patrimonio digitale. Musei, Archivi, Bibliotec...",https://www.uniba.it/corsi/patrimonio-digitale...,LM-5\nLM-43,in presenza,Libero,Accesso con laurea,58502,BARI,www.uniba.it
2,https://www.universitaly.it/index.php/scheda/s...,"BIOLOGIA AMBIENTALE , BARI\n[scheda completa (...",https://manageweb.ict.uniba.it/ricerca/diparti...,LM-6,in presenza,Libero,Accesso con laurea,59187,BARI,www.uniba.it
3,https://www.universitaly.it/index.php/scheda/s...,"BIOLOGIA CELLULARE E MOLECOLARE , BARI\n[sched...",https://manageweb.ict.uniba.it/ricerca/diparti...,LM-6,in presenza,Libero,Accesso con laurea,59183,BARI,www.uniba.it
4,https://www.universitaly.it/index.php/scheda/s...,"SCIENZE BIOSANITARIE , BARI\n[scheda completa ...",https://manageweb.ict.uniba.it/ricerca/diparti...,LM-6,in presenza,Libero,Accesso con laurea,57841,BARI,www.uniba.it
...,...,...,...,...,...,...,...,...,...,...
2387,https://www.universitaly.it/index.php/scheda/s...,"Data Science , VERONA\n[scheda completa (SUA-C...",https://www.scienzeingegneria.univr.it/?ent=cs...,LM-91,in presenza,Libero,Accesso con laurea,56113,VERONA,www.univr.it
2388,https://www.universitaly.it/index.php/scheda/s...,"Scienze infermieristiche e ostetriche , BOLZAN...",https://www.medicina.univr.it/?ent=cs&id=469,LM/SNT1,in presenza,Numero programmato,Accesso con laurea,56114,VERONA,www.univr.it
2389,https://www.universitaly.it/index.php/scheda/s...,Scienze riabilitative delle professioni sanita...,https://www.medicina.univr.it/?ent=cs&id=470,LM/SNT2,in presenza,Numero programmato,Accesso con laurea,56115,VERONA,www.univr.it
2390,https://www.universitaly.it/index.php/scheda/s...,Scienze delle professioni sanitarie tecniche a...,https://www.corsi.univr.it/?ent=cs&id=1043,LM/SNT3,in presenza,Numero programmato,Accesso con laurea,56546,VERONA,www.univr.it


In [21]:
#checking step: run only if needed
'''
df[df['ateneo_url'] == 'check corso_url'].to_csv('exception_ateneo.csv')
df.to_csv('master_3_v3.csv')
'''
df.iloc[:,13]

0       Università degli Studi di BARI ALDO MORO
1       Università degli Studi di BARI ALDO MORO
2       Università degli Studi di BARI ALDO MORO
3       Università degli Studi di BARI ALDO MORO
4       Università degli Studi di BARI ALDO MORO
                          ...                   
2387            Università degli Studi di VERONA
2388            Università degli Studi di VERONA
2389            Università degli Studi di VERONA
2390            Università degli Studi di VERONA
2391            Università degli Studi di VERONA
Name: nome_atenei, Length: 2392, dtype: object

In [9]:
df_atenei = pd.read_csv('dati_university.csv')

In [10]:
#merge data set

df_merge = pd.merge(df, df_atenei, how='left', left_on= df['ateneo_url'], right_on =  df_atenei['website'])

testing_subset = df_merge[df_merge['nome_atenei'].isnull()]

#184 record da controllare

In [14]:
#data quality: review inconsistencies
len(df[df['nome_atenei'].isnull()])

df[df['nome_atenei'].isnull()].groupby('ateneo_url')['Link'].count()

ateneo_url
check corso_url                     29
www.bionicsengineering.it            1
www.biotechuniroma2.it               1
www.campuslucca.it                   1
www.cineca.it                       17
www.difesa.it                        2
www.dstunisannio.it                  4
www.emanagementunisannio.it          1
www.ingegneriachimicapisa.it         1
www.iuav.it                          6
www.laureesanitarietorvergata.it     1
www.na.it                            7
www.statisticaunisannio.it           1
www.unidarc.it                       2
www.unina2.it                        1
www.unior.it                         9
www.uniroma5.it                      3
www.unisalento.it                   33
www.unitelmasapienza.it              3
www.unitus.it                       18
www.universitaeuropeadiroma.it       2
www.universitaly.it                 41
Name: Link, dtype: int64

In [15]:
## gestione delle eccezioni
#'Politecnico di BARI'
df.loc[df['ateneo_url'] == 'www.cineca.it',['nome_atenei']] = 'Politecnico di BARI'

#'Universitá Orientale di NAPOLI'
df.loc[df['ateneo_url'] == 'www.unior.it',['nome_atenei']] = 'Universitá Orientale di NAPOLI'

#'Università degli Studi Suor Orsola Benincasa - NAPOLI'
df.loc[df['ateneo_url'] == 'www.na.it',['nome_atenei']] = 'Università degli Studi Suor Orsola Benincasa - NAPOLI'

#Università Telematica San Raffaele Roma
df.loc[df['ateneo_url'] == 'www.uniroma5.it',['nome_atenei']] = 'Università Telematica San Raffaele - ROMA'

#Università degli Studi della TUSCIA
df.loc[df['ateneo_url'] == 'www.unitus.it',['nome_atenei']] = 'Università degli Studi della TUSCIA'

#Università Iuav di Venezia
df.loc[df['ateneo_url'] == 'www.iuav.it',['nome_atenei']] = 'Università Iuav di Venezia'

#Università del SALENTO
df.loc[df['ateneo_url'] == 'www.unisalento.it',['nome_atenei']] = 'Università del SALENTO'

#Università degli Studi di Roma UnitelmaSapienza
df.loc[df['ateneo_url'] == 'www.unitelmasapienza.it',['nome_atenei']] = 'Università degli Studi di Roma UnitelmaSapienza'

#Università Europea di Roma
df.loc[df['ateneo_url'] == 'www.universitaeuropeadiroma.it',['nome_atenei']] = 'Università Europea di Roma'

#Accademia Navale Livorno
df.loc[df['ateneo_url'] == 'www.difesa.it',['nome_atenei']] = 'Accademia Navale Livorno'

#Università degli Studi del SANNIO di BENEVENTO
df.loc[df['ateneo_url'] == 'www.dstunisannio.it',['nome_atenei']] = 'Università degli Studi del SANNIO di BENEVENTO'
df.iloc[1938:1940,8] = 'Università degli Studi del SANNIO di BENEVENTO'

#Università per stranieri Dante Alighieri di Reggio Calabria
df.loc[df['ateneo_url'] == 'www.unidarc.it',['nome_atenei']] = 'Università per stranieri Dante Alighieri di Reggio Calabria'

#UNINETTUNO
df.iloc[2025:2030,8] = 'Università Telematica Internazionale UNINETTUNO'

#Università Telematica "E-CAMPUS"
df.iloc[2001:2010,8] = 'Università Telematica "E-CAMPUS"'

#UNINT - Università degli studi Internazionali di Roma
df.iloc[1839:1844,8] = 'UNINT - Università degli studi Internazionali di Roma'

In [16]:
#qualche link corrotto non permette il merge per alcuni corsi in mezzo ad altri corsi validi della stessa universitá
#usando il metodo bfill, andiamo a riempire gli spazi vuoti nella colonna "nome_atenei"
df['nome_atenei'] = df['nome_atenei'].fillna(method = 'bfill')

In [19]:
#salvataggio dati: Master File
df[['Link','descrizione','LM_CODE','PDF_ID','Cittá','nome_atenei','Presenza','Numero_Chiuso','Accesso']].to_csv('Master_4_v1.csv')

#salvataggio lista PDF_ID 
#df_number = pd.DataFrame({'number_id':numbers})
#df_number.to_csv('number_id_list.csv')